In [ ]:
from query.models import Video
import cv2
import os
import datetime

# Import videos from folder

In [ ]:
def import_video(path):
    cap = cv2.VideoCapture(path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if 'CS' in path:
        m, d, y = path.split('_')[2:5]
        time = datetime.datetime(int(y), int(m), int(d))
    else:
        time = datetime.datetime(int(2019), int(1), int(1))
    print(path, num_frames, fps, width, height, time)
    Video(path=path, num_frames=num_frames, fps=fps, width=width, height=height, time=time).save()
    return Video.objects.filter(path=path)[0]

def import_frame(video):
    Frame.objects.bulk_create([Frame(number=i, video=video) for i in range(video.num_frames)])

In [ ]:
import_video('/app/data/videos/sample-clip.mp4')

In [ ]:
for video_name in sorted(os.listdir('/app/data/videos/')):
    if video_name[0] == '.':
        continue
    videos = Video.objects.filter(path__contains=video_name)
    if len(videos) > 0:
        continue
    print(video_name)
    video = import_video(os.path.join('/app/data/videos/', video_name))
    import_frame(video)

In [ ]:
video = Video.objects.filter(path__contains='table')[0]

In [ ]:
video.path = '/app/data/videos/Tabletennis_2012_Olympics_men_single_final_gold.mp4'
video.save()

# Analyze proportion of frames with faces

In [ ]:
videos = Video.objects.filter(path__contains='CS248').order_by('id')
for video in videos:
    frames_with_face = Frame.objects.filter(video_id=video.id, tags__name='mtcnn:labeled')
    print(video.item_name())
    print(1. * len(frames_with_face) / video.num_frames)

# Plot face distribution

In [ ]:
from rekall.interval_list import IntervalList
from esper.rekall import *
from esper.prelude import esper_widget

In [ ]:
face_intrvllists = {}
for video in videos:
    faces = Face.objects.filter(frame__video_id=video.id).order_by('frame__number')
    if len(faces) > 0:
        face_intrvllists[video.id] = IntervalList([(f.frame.number, f.frame.number+1, 0) for f in faces]).coalesce()

In [ ]:
esper_widget(intrvllists_to_result(face_intrvllists), use_jupyter_keybindings=True)